In [1]:
#Deploys the Account Detail smart contract
from new_helper import *

print(40*"-")

id = "80080"
DOMAIN = "test"

project_id = f"{id}@{DOMAIN}"

print(project_id)

----------------------------------------
80080@test


In [2]:
# Option 10 - Integrated with detail setting

from new_helper import *
from super_helper import *

directory_path = "upload"

schema = get_schema() #super_helper.py

print(schema)

processed_data = process_files(directory_path, project_id, schema) #new_helper.py
 
    

2024-12-25 19:54:23.388 | ERROR    | super_helper:index_metadata:124 - Missing required metadata keys: name, size


<Schema: ['abstract', 'cid', 'created', 'creator', 'date', 'description', 'filetype', 'format', 'full_text', 'language', 'modified', 'name', 'project_id', 'publisher', 'size', 'subject', 'title']>
file path:  upload/Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf
file name:  Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf
metadata : {'pdf:pdfversion': '1.7', 'pdf:docinfo:title': 'Aeroacoustic airfoil shape optimization enhanced by autoencoders', 'xmp:creatortool': 'Elsevier', 'pdf:hasxfa': 'false', 'access_permission:can_print_degraded': 'true', 'x-tika:content_handler': 'ToTextContentHandler', 'pdf:num3dannotations': '0', 'dc:format': 'application/pdf; version=1.7', 'pdf:docinfo:custom:robots': 'noindex', 'pdf:docinfo:creator_tool': 'Elsevier', 'access_permission:fill_in_form': 'true', 'pdf:docinfo:custom:creationdate--text': '7th February 2023', 'pdf:hascollection': 'false', 'pdf:encrypted': 'false', 'dc:title': 'Aeroa

In [3]:
#Query - GetAccountDetail

query = iroha.query('GetAccountDetail',account_id=f"{id}@{DOMAIN}")
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response.account_detail_response
details = data.detail
# ic(details)
print(f'Project Account id = {id}@{DOMAIN}, {details}')


Project Account id = 80080@test, { "admin@test" : { "file_1" : "QmXh37WXcrLXkJX2cAPjbPdKZ2cxJXD58XX6eU1Zc41ka4, QmWcUWh7h2FHLj4LhTb8ievRdvSBf9fnhMhQ7taYdQo8iT", "file_1_CID" : "QmXh37WXcrLXkJX2cAPjbPdKZ2cxJXD58XX6eU1Zc41ka4", "file_1_metadata_CID" : "Qmekwdsnu16Ka3B4Giycyp81GKrbyAXbPUArcHeBJgFgir", "linked_user" : "recursing_cohen@test", "project_metadata_cid" : "QmNz4Sc1U7zW992mvHPLtPLzS3QvGJoYWfBfSUh8TUpsHM" } }


In [4]:
# Example search usage
ix = setup_index(schema) #super_helper.py
print(ix)

keyword = "airfoil"
search_index(keyword, ix) #super_helper.py


2024-12-25 19:54:29.125 | INFO     | super_helper:setup_index:260 - Opened existing index.


FileIndex(FileStorage('indexdir'), 'MAIN')
results : <Top 1 Results for Term('full_text', 'airfoil') runtime=0.0007549459114670753>
result : <Hit {'abstract': '', 'cid': '', 'created': '', 'creator': '', 'date': '', 'description': '', 'filetype': '', 'format': '', 'language': '', 'modified': '', 'name': '', 'project_id': '', 'publisher': '', 'size': 0, 'subject': '', 'title': ''}>


[]

In [5]:
print(ix)
project_ids = search_index(keyword, ix)
print(project_ids)  # prints the list of project ids found in the results

FileIndex(FileStorage('indexdir'), 'MAIN')
results : <Top 1 Results for Term('full_text', 'airfoil') runtime=0.0007646498270332813>
result : <Hit {'abstract': '', 'cid': '', 'created': '', 'creator': '', 'date': '', 'description': '', 'filetype': '', 'format': '', 'language': '', 'modified': '', 'name': '', 'project_id': '', 'publisher': '', 'size': 0, 'subject': '', 'title': ''}>
[]
